In [1]:
import os
from distutils.dir_util import copy_tree

import geopandas as gpd
import nivapy3 as nivapy
import pandas as pd
import teotil_report as rep

In [2]:
eng = nivapy.da.connect()

Username:  ········
Password:  ········


Connection successful.


# Summarise TEOTIL2 results

This notebook aggregates TEOTIL results files to match groupings used in the annual TEOTIL reports.

## 1. Create results tables for report regions

Generate one CSV file for each table in the report. **Based on the data hosted on GitHub - be sure to push new model results to the TEOTIL repository before running this code**. 

In [3]:
st_yr = 1990
end_yr = 2021

out_fold = f"../report_{end_yr}/data"

In [4]:
df = rep.get_teotil_results_main_catchments(st_yr, end_yr)
result_dict = {}
for par in ["n", "p"]:
    agg_par_df = rep.aggregate_parameters(df, par)
    par_res_dict = rep.aggregate_regions(agg_par_df, par, out_fold)
    result_dict[par] = par_res_dict

## 2. Map data

Get data from the database and export as shapefiles for use with ArcGIS templates.

In [5]:
year = 2021

### 2.1. Site maps

#### 2.1.1. Copy ArcGIS templates

In [6]:
gis_fold = f"../report_{year}/gis"
if not os.path.exists(gis_fold):
    os.makedirs(gis_fold)

copy_tree("../arcgis_templates", gis_fold)
vec_fold = f"../report_{year}/gis/vector"

#### 2.1.2. Export industry and sewage treatment data

In [7]:
sql = (
    """
select * from RESA2.RID_PUNKTKILDER
where anlegg_nr in (
select anlegg_nr from RESA2.RID_PUNKTKILDER_INPAR_VALUES
where year = %s
and inp_par_id in (26, 27, 34, 35, 44, 45))
"""
    % year
)
df = pd.read_sql(sql, eng)
df.dropna(subset=["lat_utl", "lon_utl"], inplace=True)

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon_utl, df.lat_utl, crs="epsg:4326")
).to_crs("epsg:25833")
gdf.head()

,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status,geometry
0,1149.0036.01,FMC BIOPOLYMER AS,SFT,None,None,INDUSTRI,1149,Vormedalsvegen 301,5545.0,039.80,5.318689,59.354188,5.318689,59.354188,Aktiv,POINT (-49223.967 6619557.996)
1,1520.0008.01,Ørsta Stål AS,MVAMR,None,None,INDUSTRI,1520,Skorgeura,6150.0,095.210,6.043068,62.213521,6.043068,62.213521,Aktiv,POINT (35224.017 6930200.998)
2,1531.0011.01,Vedde Sildeoljefabrikk AS,MVAMR,None,None,INDUSTRI,1531,Brevika Industriv 4,6018.0,101.41,6.241718,62.436091,6.241718,62.436091,Aktiv,POINT (48846.014 6953415.999)
3,1535.0001.01,"TINE MIDT-NORGE BA, Tresfjord",MVAMR,None,None,INDUSTRI,1535,None,6391.0,102.61,7.128080,62.525529,7.128080,62.525529,Aktiv,POINT (95544.005 6957432.999)
4,1812.0003.01,"TINE NORD-NORGE, Sømna",MVANO,None,None,INDUSTRI,1812,None,8920.0,146.310,12.205149,65.360168,12.205149,65.360168,Aktiv,POINT (370015.000 7251478.000)


In [8]:
ind_gdf = gdf.query("type == 'INDUSTRI'")
stan_gdf = gdf.query("type == 'RENSEANLEGG'")

ind_gdf.to_file(os.path.join(vec_fold, "industry.shp"))
stan_gdf.to_file(os.path.join(vec_fold, "sewage.shp"))

/tmp/ipykernel_7818/768295948.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ind_gdf.to_file(os.path.join(vec_fold, "industry.shp"))
/tmp/ipykernel_7818/768295948.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  stan_gdf.to_file(os.path.join(vec_fold, "sewage.shp"))


In [9]:
nivapy.spatial.quickmap(ind_gdf, lon_col="lon_utl", lat_col="lat_utl")

In [10]:
nivapy.spatial.quickmap(stan_gdf, lon_col="lon_utl", lat_col="lat_utl")

#### 2.1.3. Export aquaculture data

In [11]:
sql = (
    """
select * from RESA2.RID_KILDER_AQUAKULTUR
where nr in (
select anlegg_nr from RESA2.RID_KILDER_AQKULT_VALUES
where ar = %s)
"""
    % year
)

aq_df = pd.read_sql(sql, eng)
aq_df = aq_df.query("bredde < 30")
aq_gdf = gpd.GeoDataFrame(
    aq_df,
    geometry=gpd.points_from_xy(aq_df.bredde, aq_df.lengde, crs="epsg:4326"),
).to_crs("epsg:25833")
del aq_gdf["midl"]
aq_gdf.head()

,nr,navn,etablert,type,komnr,plassering,miljo,kap,mh,lengde,bredde,regine,geometry
0,11272,ØKSENGÅRD,2004.0,MIDLERTIDIG,1840,SJØ,SALTVANN,2700.0,TN,67.137098,15.405270,163.31,POINT (517571.094 7446720.800)
1,11303,VEGGFJELL,1984.0,MIDLERTIDIG,1849,SJØ,SALTVANN,5400.0,TN,67.960650,15.805867,170.22,POINT (533745.515 7538697.112)
2,11312,SVARTFJELL,1984.0,MIDLERTIDIG,1848,SJØ,SALTVANN,3600.0,TN,67.939483,15.503168,169.52,POINT (521089.570 7536203.040)
3,11315,VINKFJORDEN,1983.0,MIDLERTIDIG,1848,SJØ,SALTVANN,3600.0,TN,67.726378,15.379890,168.21,POINT (516068.496 7512407.687)
4,11320,OKSØY,1984.0,MIDLERTIDIG,1848,SJØ,SALTVANN,5400.0,TN,67.974021,15.308233,169.7,POINT (512900.020 7540000.041)


In [12]:
aq_gdf.to_file(os.path.join(vec_fold, "aquaculture.shp"))

#### 2.1.4. Make site maps

After running the cells above, open the ArcMap documents for industry, sewage and aquaculture in `../report_{year}/gis` and export the maps to `../report_{year}/plots` as files named e.g. `aquaculture.png`. Remember to **clip the output to the map extent**.

### 2.2. Summary fluxes for regions

#### 2.2.1. Summaries for Vannregioner

In [13]:
vannreg_list = [
    "Glomma",
    "Vest-Viken",
    "Agder",
    "Rogaland",
    "Hordaland",
    "Sogn og Fjordane",
    "Møre og Romsdal",
    "Trøndelag",
    "Nordland",
    "Troms",
    "Finnmark",
]

for par in ["n", "p"]:
    gdf = gpd.read_file(r"../arcgis_templates/vector/Vannregioner.shp")
    df_list = []
    for reg in vannreg_list:
        fpath = f"../report_{year}/data/{reg}_{par}.csv"
        df = pd.read_csv(fpath)
        df = df.query("År == @year")
        df["Vannreg"] = reg
        df.drop(["År", "Menneskeskapt"], axis="columns", inplace=True)
        df_list.append(df)

    df = pd.concat(df_list, axis="rows")
    gdf = gdf.merge(df, how="left", on="Vannreg")
    gdf.to_file(os.path.join(vec_fold, f"{par}_by_vannreg.shp"))

#### 2.2.2. Summaries for Forvaltningsplanområder

In [14]:
forvaltom_list = [
    "Nordsjøen",
    "Norskehavet",
    "Barentshavet",
]

for par in ["n", "p"]:
    gdf = gpd.read_file(r"../arcgis_templates/vector/Forvaltningsplanområder.shp")
    df_list = []
    for reg in forvaltom_list:
        fpath = f"../report_{year}/data/{reg}_{par}.csv"
        df = pd.read_csv(fpath)
        df = df.query("År == @year")
        df["Hav"] = reg
        df.drop(["År", "Menneskeskapt"], axis="columns", inplace=True)
        df_list.append(df)

    df = pd.concat(df_list, axis="rows")
    gdf = gdf.merge(df, how="left", on="Hav")
    gdf.to_file(os.path.join(vec_fold, f"{par}_by_forvaltom.shp"))

#### 2.2.3. Make pie chart maps

After running the cells above, open the other four ArcMap documents in `../report_{year}/gis` and export the maps to `../report_{year}/plots` as files named e.g. `forvaltom_n.png`. Remember to **clip the output to the map extent**.